# Introduction
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-block alert-warning">
<font color=black>

**What?** Errors and exceptions

</font>
</div>

# Errors and Exceptions
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=black>
    
- There are three type of errors:

    - **Syntax errors:** Errors where the code is not valid Python (generally easy to fix)
    - **Runtime errors:** Errors where syntactically valid code fails to execute, perhaps due to invalid user input (sometimes easy to fix)
    - **Semantic errors:** Errors in logic: code executes without a problem, but the result is not what you expect (often very difficult to track-down and fix)


- Here we're going to focus on how to deal cleanly with *runtime errors*.
- As we'll see, Python handles runtime errors via its *exception handling* framework.

</font>
</div>

## Runtime Errors

If you've done any coding in Python, you've likely come across runtime errors.
They can happen in a lot of ways.

For example, if you try to reference an undefined variable:

In [3]:
print(Q)

NameError: name 'Q' is not defined

Or if you try an operation that's not defined:

In [4]:
1 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Or you might be trying to compute a mathematically ill-defined result:

In [5]:
2 / 0

ZeroDivisionError: division by zero

Or maybe you're trying to access a sequence element that doesn't exist:

In [6]:
L = [1, 2, 3]
L[1000]

IndexError: list index out of range

Note that in each case, Python is kind enough to not simply indicate that an error happened, but to spit out a *meaningful* exception that includes information about what exactly went wrong, along with the exact line of code where the error happened.
Having access to meaningful errors like this is immensely useful when trying to trace the root of problems in your code.

# try and except
<hr style="border:2px solid black"> </hr>

<div class="alert alert-info">
<font color=black>

- The main tool Python gives you for handling runtime exceptions is the try...except clause. Its basic structure is this: 

</font>
</div>

In [7]:
try:
    print("this gets executed first")
except:
    print("this gets executed only if there is an error")

this gets executed first


Note that the second block here did not get executed: this is because the first block did not return an error.
Let's put a problematic statement in the ``try`` block and see what happens:

In [8]:
try:
    print("let's try something:")
    x = 1 / 0 # ZeroDivisionError
except:
    print("something bad happened!")

let's try something:
something bad happened!


Here we see that when the error was raised in the ``try`` statement (in this case, a ``ZeroDivisionError``), the error was caught, and the ``except`` statement was executed.

One way this is often used is to check user input within a function or another piece of code.
For example, we might wish to have a function that catches zero-division and returns some other value, perhaps a suitably large number like $10^{100}$:

In [9]:
def safe_divide(a, b):
    try:
        return a / b
    except:
        return 1E100

In [10]:
safe_divide(1, 2)

0.5

In [11]:
safe_divide(2, 0)

1e+100

There is a subtle problem with this code, though: what happens when another type of exception comes up? For example, this is probably not what we intended:

In [12]:
safe_divide (1, '2')

1e+100

<div class="alert alert-info">
<font color=black>

- Dividing an integer and a string raises a `TypeError`, which our over-zealous code caught and assumed was a ``ZeroDivisionError``!
- For this reason, it's **nearly always a better idea** to catch exceptions *explicitly*:
- Please note that we're now catching zero-division errors only, and letting all other errors pass through un-modified.

</font>
</div>

In [13]:
def safe_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return 1E100

In [14]:
safe_divide(1, 0)

1e+100

In [15]:
safe_divide(1, '2')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

## `try`/`except` block that catches all exceptions?

<div class="alert alert-info">
<font color=black>

- Sometimes is it tedious to try to catch all the exception, so someone may be tempted to try to write a **catch all statement**.
- In most cases you are, probably, doing **something wrong** if you are trying to catch any exception. I mean you can simply misspell something in your code and you will even don't know about it. It is a **good practice** to catch specific exceptions.
- To be more precise, catching all possible exceptions is only a problem if they are **caught silently**.
- An exception to this general rule is when the caught error messages are printed to `sys.stderr` and possibly logged. That is a perfectly valid and common exception.
- A simple but not perfect solution is to use `except Exception as e:`.
    
</font>
</div>

In [ ]:
import traceback
import logging

try:
    whatever()
except Exception as e:
    logging.error(traceback.format_exc())
    # Logs the error appropriately.

In [ ]:
try:
    a = 2/0
except Exception as e:
    print(e.__doc__)
    print(e.message)

<div class="alert alert-info">
<font color=black>

- The advantage of except Exception over the bare except is that there are a few exceptions that it wont catch, most obviously `KeyboardInterrupt` and `SystemExit`. 

</font>
</div>

# Raising Exceptions
<hr style="border:2px solid black"> </hr>

<div class="alert alert-info">
<font color=black>

- The way you raise your **OWN exceptions** is with the raise statement. 
- We can call `raise` directly as in `raise RuntimeError("my error message")` to see how it works.
- Keep in mind that `raise` is a python name recognised by python but `RuntimeError` is something you decide. For instance,  something like `value_error` would have been equally OK. 

</font>
</div>

In [4]:
# Let start with a very simple example
raise RuntimeError("my error message")

RuntimeError: my error message

In [6]:
# We'd like to let users know that only positive numbers are allowed!
def fibonacci(N):
    if N < 0:
        raise ValueError("N must be non-negative")
    L = []
    a, b = 0, 1
    while len(L) < N:
        a, b = b, a + b
        L.append(a)
    return L

In [7]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [8]:
fibonacci(-10)

ValueError: N must be non-negative

<div class="alert alert-info">
<font color=black>

- There is another option: we could even use a try...except block to handle it!

</font>
</div>

In [21]:
N = -10
try:
    print("trying this...")
    print(fibonacci(N))
except ValueError:
    print("Bad value: need to do something else")

trying this...
Bad value: need to do something else


<div class="alert alert-info">
<font color=black>

- Let's say we'd like to **access** the error message itself.
- We can do that with this contruct `except ZeroDivisionError as err`.
- Keep in mind that python needs to know `ZeroDivisionError` exihists. If you had ued something like this `ZeroDivisionError_` it would not have worked.

</font>
</div>

In [11]:
try:
    x = 1 / 0
except ZeroDivisionError as err:
    print("Error class is:  ", type(err))
    print("Error message is:", err)

Error class is:   <class 'ZeroDivisionError'>
Error message is: division by zero


<div class="alert alert-info">
<font color=black>

- In addition to built-in exceptions, it is possible to define **custom** exceptions through *class inheritance*.
- For instance, if you want a special kind of ``ValueError``, you can do this: 

</font>
</div>

In [13]:
class MySpecialError(ValueError):
    pass

# How you'd call this
raise MySpecialError("here's the message")

MySpecialError: here's the message

This would allow you to use a ``try``...``except`` block that only catches this type of error:

In [14]:
try:
    print("do something")
    raise MySpecialError("[informative error message here]")
except MySpecialError:
    print("do something else")

do something
do something else


# `finally` clause
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-info">
<font color=black>

- A `finally` clause is always executed before leaving the `try` statement, whether an exception has occured or not.
- The `finally` clause is also executed on the way out when any other clause of the `try` statement is left via a `break`, `continue` or `return` statement.


- In the real world, the `finally` clause is used for releasing external resources that are always useful regardless of what happens inside the function.

</font>
</div>

In [2]:
def divide(x, y):
    try:
        result = x/y
    except ZeroDivisionError:
        print("Division by zero!")
    else:
        print("Result is:", result)
    finally:
        print("Executing finally clause")

In [3]:
divide(2,1)

Result is: 2.0
Executing finally clause


In [4]:
divide(2,0)

Division by zero!
Executing finally clause


In [5]:
divide("2", "1")

Executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

# References
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-block alert-warning">
<font color=black>

- [Whirlwind Tour of Python](http://www.oreilly.com/programming/free/a-whirlwind-tour-of-python.csp)
- https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions

</font>
</div>